In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d, spec1d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/LRIS_data/LRIS_red_data_20201120/keck_lris_red_B/Science/*'

In [3]:
file_list = []
out_file = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and 'J0805' in p:
        if '.fits' in p:
            file_list.append(p)
            frame = p.split('_')[-3][:4]
            out_file.append('J0805_LRIS_spectra/J0805_r'+ frame+'_cleaned.fits')

In [4]:
file_list, out_file

(['../galaxy_spectra/LRIS_data/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0043-J0805+6919_LRISr_2020Nov21T152211.856.fits'],
 ['J0805_LRIS_spectra/J0805_r0043_cleaned.fits'])

In [5]:
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[20, 200])#, transpose=True)
    spec.set_dispaxis('y')
    #spec.subtract_sky_2d(use_skymod=True)
    #spec.szap_new(infile='B0631_median_image.fits')
    spec.szap(use_skymod=True)
    
    filled_data = spec.data
    mask = spec.szap_mask
    
    wav_im = pf.open(p)[19].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    sky_im = pf.open(p)[14].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    ivar_im = pf.open(p)[13].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    #print(ivar_im.shape)
    ivar_im[mask] = 1.e-5
    
    prim_hdr = pf.open(p)[0].header #pf.Header()
    #prim_hdr.append(('OBJECT', 'J08'))
    prim_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
    phdu = pf.PrimaryHDU(header=prim_hdr)
    
    sci_hdr = pf.Header()
    sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
    sci_im_hdu = pf.ImageHDU(data=filled_data, header=sci_hdr, name='SCI_IM')
    
    ivar_hdr = pf.Header()
    ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
    ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr, name='IVAR')
    
    sky_hdr = pf.Header()
    sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
    sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr, name='SKY_MOD')
    
    wav_hdr = pf.Header()
    wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
    wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr, name='WAV_IM')
    
    mask_hdr = pf.Header()
    mask_hdr.append(('IMTYPE', 'MASK'))
    cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
        'location from median image'
    mask_hdr.append(('', cmt))
    mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr, name='MASK')
    
    hdul = pf.HDUList([phdu, sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
    hdul.writeto(out_file[i])


Loading file ../galaxy_spectra/LRIS_data/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0043-J0805+6919_LRISr_2020Nov21T152211.856.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_data/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0043-J0805+6919_LRISr_2020Nov21T152211.856.fits (HDU=12)
The input dataset was trimmed
 xrange: 20:201.  yrange: 0:4096
Final data dimensions (x y): 181 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 181


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From now cosmic ray rejected data will be used



In [33]:
spec.do_waverect(resamp_ord=5, wavext=19, outfile='check_waverect_ord_5.fits')

Dispersion : 0.800000
Found 1027 NaNs in the two-dimensional spectrum

sky subtracted and cosmic ray rejected data has been resampled along with vardata in place of the coordinateds whcih rectify the tilted wave image
